#### **Var definitions**
Change it whenever necessary

In [33]:
access_oauth_token = "token <PLACE_HOLDER>" #TODO: delete before commiting the code
jira_url = "https://issues.apache.org/jira/"
project_name = "MATH"
repo_name = "commons-math"
proj = repo_name
base_path = '/Users/lorenapacheco/Concordia/Masters/'
out = base_path + "BugReportsMining/bug-reports/"
proj_path = base_path + "open_source_repos_being_studied/" + repo_name
out_commits_dir = base_path + "BugReportsMining/commits/" + repo_name

#### **Bug reports mining**
Getting all the **Jira tickets** ids from the project that are **closed** and have the **bug issue type**

In [34]:
import os
import json
import requests
import pprint
from jira import JIRA

issue_numbers_list = []

def get_jira_closed_bug_issues(jira_url, project_name):
    issue_search_str = "project = "+ project_name + " AND issuetype = Bug AND status = Closed AND resolution in (Fixed, Done, Resolved) ORDER BY priority DESC, updated DESC"
    issues = jira.search_issues(issue_search_str, maxResults=False)
    for i in issues:
        issue_numbers_list.append(str(i))
    print(issue_numbers_list)
    print("\nNumber of issues: " + str(len(issue_numbers_list)))

jira = JIRA(server=jira_url)
get_jira_closed_bug_issues(jira_url, project_name)

['MATH-60', 'MATH-940', 'MATH-915', 'MATH-922', 'MATH-916', 'MATH-429', 'MATH-283', 'MATH-1167', 'MATH-1070', 'MATH-961', 'MATH-947', 'MATH-929', 'MATH-950', 'MATH-924', 'MATH-109', 'MATH-96', 'MATH-153', 'MATH-41', 'MATH-54', 'MATH-327', 'MATH-464', 'MATH-572', 'MATH-352', 'MATH-343', 'MATH-344', 'MATH-358', 'MATH-200', 'MATH-1166', 'MATH-1505', 'MATH-1342', 'MATH-1252', 'MATH-1246', 'MATH-1118', 'MATH-1300', 'MATH-1297', 'MATH-1251', 'MATH-1197', 'MATH-1302', 'MATH-1269', 'MATH-1237', 'MATH-1266', 'MATH-1310', 'MATH-1255', 'MATH-1303', 'MATH-1203', 'MATH-1208', 'MATH-1209', 'MATH-1211', 'MATH-1191', 'MATH-1204', 'MATH-1149', 'MATH-1154', 'MATH-1162', 'MATH-1127', 'MATH-984', 'MATH-1174', 'MATH-1148', 'MATH-1145', 'MATH-1125', 'MATH-1131', 'MATH-1129', 'MATH-1123', 'MATH-1146', 'MATH-1138', 'MATH-1181', 'MATH-1152', 'MATH-985', 'MATH-821', 'MATH-1103', 'MATH-1005', 'MATH-1019', 'MATH-1089', 'MATH-1115', 'MATH-1110', 'MATH-1107', 'MATH-1093', 'MATH-989', 'MATH-1065', 'MATH-1059', 'MATH

#### **Bug reports content extraction**
Extraction the content of each of these bug reports

In [35]:
output_folder = out + repo_name+ '/'

def get_bug_comments(issue):
    comments = issue.fields.comment.comments
    comments_list = []
    for c in comments:
        comment = {
            "author": {
                "login": c.author.key
            },
            "authorAssociation": "NONE",
            "body": c.body,
            "createdAt": c.created,
            "includesCreatedEdit": False,
            "isMinimized": False,
            "minimizedReason": "",
            "reactionGroups": []
        }
        comments_list.append(comment)
    return comments_list

def get_bug_details(issue_number):
    issue = jira.issue(issue_number)
    bug_details = {
        "title": issue.fields.summary,
        "body": issue.fields.description,
        "comments_content": get_bug_comments(issue),
        "createdAt": issue.fields.created,
        "id": issue_number,
        "labels":[
            {
                "name": issue.fields.issuetype.name,
                "id": issue.fields.issuetype.id,
                "description": issue.fields.issuetype.description,
                "color": None
            }
        ],
        "number": issue_number,
        "state": issue.fields.status.name,
        "updatedAt": issue.fields.updated,
        "url": jira_url + "browse/"+ issue_number
    }
    return bug_details
    

def dict_to_json_file(file, dic, folder=output_folder):
    if not os.path.exists(folder):
        os.makedirs(folder)
    with open(os.path.join(folder, file+'.json'), 'w') as fp:
        json.dump(dic, fp, sort_keys=True, indent=4)
    fp.close()
        
for issue_number in issue_numbers_list:
    bug_details = get_bug_details(issue_number)
    dict_to_json_file(repo_name+'-'+str(issue_number), bug_details)
print ("Extraction completed")
    

Extraction completed


#### **Filtering for logs and stack traces**
Filtering only the bug reports with **logs** and **stack traces**

In [36]:
import re
import glob
from collections import defaultdict

# global dict of a dict for storing the log
bug_report_log = defaultdict(dict)
bug_report_stack_trace = defaultdict(dict)

def json_file_to_dict(file):
    data = {}
    with open(os.path.join(output_folder, file+'.json'), 'r') as fp:
        data = json.load(fp)
    fp.close()
    return data

def find_regex(regex, var,file_name, text_content):
    results = re.finditer(regex, text_content)
    if results:
        for log in results:
            bug_name = file_name.split('-')[-2] + '-' + file_name.split('-')[-1]
            var[proj][bug_name] = log.group() 
            
            
def find_logs_and_stack_traces (file_name, text_content):
    stack_trace_regex = r'(?m)^.*?Exception.*(?:\n+^\s*at .*)+'
    logs_regex = r'(ERROR|INFO|WARN|DEBUG|FATAL)\s+(?P<class>\w+(\.\w+)*)'
    find_regex(logs_regex, bug_report_log,file_name, text_content)
    find_regex(stack_trace_regex, bug_report_stack_trace,file_name, text_content)
    

# for every .json in bug-reports folder
for file in glob.glob(output_folder +'/*.json'):
    file_name = os.path.basename(file).replace('.json', '')
    bug_report = json_file_to_dict(file.replace('.json', ''))
    if not bug_report:
        continue
    
    title = bug_report['title'] if bug_report['title'] else ""
    description = bug_report['body'] if bug_report['body'] else ""
    text_content = title + '\n' + description + '\n' + '\n'.join([comment['body'] for comment in bug_report['comments_content'] if 'body' in comment and comment['body']])
    find_logs_and_stack_traces(file_name, text_content)

print(proj)
print("Bug reports with logs:" + str(len(bug_report_log[proj])))
print("Bug reports with stack traces:" + str(len(bug_report_stack_trace[proj])))

commons-math
Bug reports with logs:1
Bug reports with stack traces:22


#### **Commits search**
Searching for the respective commits

**TODO:** Figure out a way to decide witch of the commits to use

In [37]:
from urllib.parse import quote

bug_reports_with_logs_commits = defaultdict(dict)
bug_reports_with_stack_traces_commits = defaultdict(dict)
cont = 0

def get_bug_report_commit(bug, proj_path):
    os.chdir(proj_path)
    log_grep_command = "git log --grep=\""+ str(bug)+"[^0-9]\" --grep=\""+ str(bug)+"$\" --pretty=format:\"%H\" >> commit_output"
    os.system(log_grep_command)
    lines = []
    with open("commit_output", 'r') as fp:
        lines =fp.readlines()
        fp.close()
    os.system("rm commit_output")
    if (lines):
        commit_hash = lines[0]
        return(commit_hash)
    return ""

for bug in bug_report_log[proj].keys():
    commit = get_bug_report_commit(bug, proj_path)
    if (commit != ""):
        previous_commit_command = "git rev-parse " + commit.strip() + "~1  >> previous_commit_output"
        os.system(previous_commit_command)
        previous_commit = ""
        with open('previous_commit_output', 'r') as fp:
            previous_commit = fp.read().replace('\n', '')
            fp.close()
        os.system("rm previous_commit_output")
        content = {
            "log": bug_report_log[proj][bug],
            "commit": commit.strip(), 
            "previous_commit": previous_commit
        }
        bug_reports_with_logs_commits[bug] = content
        cont = cont + 1

for bug in bug_report_stack_trace[proj].keys():
    commit = get_bug_report_commit(bug, proj_path)
    if (commit != ""):
        previous_commit_command = "git rev-parse " + commit.strip() + "~1  >> previous_commit_output"
        os.system(previous_commit_command)
        previous_commit = ""
        with open('previous_commit_output', 'r') as fp:
            previous_commit = fp.read().replace('\n', '')
            fp.close()
        os.system("rm previous_commit_output")
        content = {
            "stack_trace": bug_report_stack_trace[proj][bug],
            "commit": commit.strip(), 
            "previous_commit": previous_commit
        }
        bug_reports_with_stack_traces_commits[bug] = content
        cont = cont + 1

if bug_reports_with_logs_commits:
    dict_to_json_file("bug_reports_with_logs_commits", bug_reports_with_logs_commits, out_commits_dir)
if bug_reports_with_stack_traces_commits:
    dict_to_json_file("bug_reports_with_stack_traces_commits", bug_reports_with_stack_traces_commits, out_commits_dir)

print("Extraction completed")
print("Number obtained: " + str(cont))

Extraction completed
Number obtained: 17
